# Objetivo:
Neste notebook, iremos explorar as diferentes formas de **validar um modelo de Machine Learning**, ou seja, definir se um modelo realmente é eficiente, evitando tomar essa decisão baseada em algum dado enviesado. Faremos isso usando o seguinte problema:

A partir de dados sobre carros a venda, queremos prever se eles serão vendidos ou não.

#Dados:
Cada linha representa um carro que possui quatro atributos:

* **vendido:** 1 se o carro foi vendido, 0 caso não tenha sido vendido.
* **idade_do_modelo:** Tempo em anos desde o lançamento do carro
* **km_por_ano:** Média de quilômetros que o carro percorreu a cada ano
* **preco:** O preço que foi cobrado pelo carro

In [16]:
import pandas as pd

uri = "https://raw.githubusercontent.com/caalvaro/machine-learning/main/Classification%20-%20Cross%20Validation/venda_de_carros.csv"
car_sales = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
car_sales.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


# Usando apenas o train_test_split
A primeira estratégia para analisar a eficiência de um modelo é separar os dados em **treino** e **teste**. Com isso, usaremos os dados de treino para treinar o modelo e os de teste, ou seja, dados novos que o modelo não conhecia antes, para avaliar a acurácia dele.

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

features = car_sales[["preco", "idade_do_modelo", "km_por_ano"]]
target = car_sales["vendido"]

SEED = 158020
np.random.seed(SEED)
train_x, test_x, train_y, test_y = train_test_split(features, target, test_size = 0.25,
                                                         stratify = target)

print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(train_x), len(test_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


Para avaliar o quão boa é a acurácia de um modelo, precisamos de alguma referência. Portanto usamos o DummyClassifier, um classificador "burro" (não usa as features para fazer a classificação) que nos dará uma *baseline* para a acurácia.

In [3]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(train_x, train_y)
accuracy = dummy_stratified.score(test_x, test_y) * 100

print("A acurácia do dummy stratified foi %.2f%%" % accuracy)

A acurácia do dummy stratified foi 58.00%


Vamos, então, avaliar a acurácia de um modelo mais robusto, como o DecisionTreeClassifier

In [4]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=2)
model.fit(train_x, train_y)
predictions = model.predict(test_x)

accuracy = accuracy_score(test_y, predictions) * 100

print("A acurácia foi %.2f%%" % accuracy)

A acurácia foi 71.92%


Vemos que a acurácia foi consideravelmente maior (cerca de 24% maior). Mas o que isso realmente significa que um modelo é melhor que o outro? Veremos.

## Limitações do train_test_split
Quando avaliamos a eficiência do modelo usando o conjunto de teste, a intenção é ver como o modelo se comporta com dados que ele ainda não viu. O problema é que essa acurácia não necessariamente vai se refletir quando o modelo estiver funcionando em produção, já que não sabemos quais dados serão enviados para ele.

**E se tivermos dado uma sorte grande do modelo ter uma facilidade maior para classificar os dados utilizados no teste?**

Isso é completamente possível. Portanto, não podemos confiar em uma métrica exclusivamente. É aí que entra a **validação cruzada**.

# Validação cruzada



Em vez de confiarmos exclusivamente no resultado de apenas um conjunto de teste, podemos fazer a divisão entre treino e teste diversas vezes. Então, usando a **média** dos resultados, teremos uma estimativa mais robusta da acurácia. Além disso, podemos também analisar o **desvio padrão** das acurácias, determinando, assim, um **intervalo de confiança** para a acurácia do modelo.

Dessa forma, estaremos menos suscetível a um enviesamento dos dados de teste, pois faremos o teste com todos os dados.

In [5]:
def print_results(results, nsplits):
  mean = results['test_score'].mean()
  standard_dev = results['test_score'].std()
  print("Accuracy com cross validation de %d splits: entre [%.2f, %.2f]. \n\
        Média = %.2f \n\
        Desvio Padrão = %.2f" \
        % (nsplits, (mean - 2 * standard_dev)*100, (mean + 2 * standard_dev) * 100, \
           mean*100, standard_dev*100))

In [6]:
from sklearn.model_selection import cross_validate

model = DecisionTreeClassifier(max_depth=2)

n_splits = 10
results = cross_validate(model, features, target, cv = n_splits, return_train_score=False)

print_results(results, n_splits)

Accuracy com cross validation de 10 splits: entre [74.24, 77.32]. 
        Média = 75.78 
        Desvio Padrão = 0.77


Agora temos mais dados para avaliar o modelo. Temos a média e o desvio padrão dos treinamentos, gerando um intervalo de confiança para a acurácia.

# Importância da aleatoriedade no cross validate

## Simulando uma situação de azar

Podemos nos deparar com uma situação os os dados estão numa ordem que prejudique o treinamento. Por exemplo, se treinarmos um modelo usando uma maioria de dados cuja classificação é '0', deixaremos o modelo enviesado por eles, o que prejudica na avaliação.

Portando, torna-se necessário garantir a aleatoriedade na hora de dividir os dados para treino e teste, assim teremos a expectativa dos conjuntos terem uma distribuição mais uniforme dos dados.

In [7]:
# ordenando os dados para que os carros não vendidos fiquem no início do dataframe

car_sales_sorted = car_sales.sort_values("vendido", ascending=True)
features_sorted = car_sales_sorted[["preco", "idade_do_modelo","km_por_ano"]]
target_sorted = car_sales_sorted["vendido"]
car_sales_sorted.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [8]:
from sklearn.model_selection import KFold

n_splits = 10
cross_validator = KFold(n_splits = n_splits)
model = DecisionTreeClassifier(max_depth=2)

results = cross_validate(model, features_sorted, target_sorted, cv = cross_validator, return_train_score=False)

print_results(results, n_splits)

Accuracy com cross validation de 10 splits: entre [34.29, 81.39]. 
        Média = 57.84 
        Desvio Padrão = 11.77


Podemos ver acima que o o resultado dos treinos variou muito. Justamente por conta do viés adicionado em alguns modelos pelos dados não distribuídos corretamente. A média da acurácia ficou baixa e o desvio padrão ficou enorme, mostrando a variação de acurácia em cada treino.

Então, forçaremos a aleatoriedade na divisão dos dados usando o parâmentro `shuffle = True` no KFold.

In [9]:
from sklearn.model_selection import KFold

n_splits = 10
cross_validator = KFold(n_splits = n_splits, shuffle = True)
model = DecisionTreeClassifier(max_depth=2)

results = cross_validate(model, features_sorted, target_sorted, cv = cross_validator, return_train_score=False)

print_results(results, n_splits)

Accuracy com cross validation de 10 splits: entre [72.91, 78.65]. 
        Média = 75.78 
        Desvio Padrão = 1.43


Podemos ver que o modelo teve um desempenho geral muito maior, com uma média mais alta e o desvio padrão mais baixo, mostrando que os modelos foram menos afetados pelo desbalanceamento dos dados no treinamento e teste.

In [10]:
from sklearn.model_selection import StratifiedKFold

n_splits = 10
cross_validator = StratifiedKFold(n_splits = n_splits, shuffle = True)
model = DecisionTreeClassifier(max_depth=2)

results = cross_validate(model, features_sorted, target_sorted, cv = cross_validator, return_train_score=False)

print_results(results, n_splits)

Accuracy com cross validation de 10 splits: entre [73.90, 77.66]. 
        Média = 75.78 
        Desvio Padrão = 0.94


Acima, temos uma estratégia ainda mais inteligente. Usando o StratifiedKFold, garantimos que a proporção entre as classes se mantenha em cada divisão feita.

O resultado disso é uma média muito próxima da anterior, mas com o desvio padrão menor, mostrando que a acurácia do modelo variou menos em cada treinamento e teste.

# Cross validation com StandardScaler

Até então, usamos um modelo de aprendizado que não é sensível à escala dos dados. Quando precisamos usar um, como o SVC, temos que ter um cuidado maior na hora de fazer a validação cruzada.

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

scaler = StandardScaler()
scaler.fit(train_x)
train_x_scaled = scaler.transform(train_x)
test_x_scaled = scaler.transform(test_x)

model = SVC()
model.fit(train_x_scaled, train_y)
previsoes = model.predict(test_x_scaled)

accuracy = accuracy_score(test_y, previsoes) * 100

print("A acurácia foi %.2f%%" % accuracy)

A acurácia foi 74.40%


Fazendo o teste do modelo usando a função train_test_split, temos que fazer a padronização dos dados após a divisão.

*Uma curiosidade é que há um erro muito comum de ser cometido nesse passp, em que se padroniza os dados antes da divisão. O problema dessa abordagem é que os dados de teste acabam sendo utilizados para calcular a média e o desvio padrão dos dados. Mas a intenção é justamente que os dados de teste sejam dados não vistos pelo modelo, então não podem ser usados para a padronização.*



Abaixo, faremos o jeito errado de padronizar os dados, ou seja, antes da divisão

In [12]:
scaler = StandardScaler()
scaler.fit(features_sorted)
features_sorted_scaled = scaler.transform(features_sorted)

In [15]:
n_splits = 10
cross_validator = StratifiedKFold(n_splits = n_splits, shuffle = True)
model = SVC()
results = cross_validate(model, features_sorted_scaled, target_sorted, cv = cross_validator, return_train_score=False)

print_results(results, n_splits)

Accuracy com cross validation de 10 splits: entre [74.57, 78.65]. 
        Média = 76.61 
        Desvio Padrão = 1.02


Mas como é o jeito certo? A divisão é feita internamente durante o cross validation, através do StratifiedKFold, então não é possível fazer isso explicitamente como no train_test_split.

Nesse caso, precisamos do uso do Pipeline, que é um encapsulamento de várias funções definidas por nós. Aqui, criaremos um Pipeline com a padronização e, em seguida, o treinamento do modelo. Dessa forma, a execução será:

1.   Dividir os dados
2.   Aplicar a padronização no conjunto de treinamento
3.   Treinar o modelo passado com o conjunto de treinamento
4.   Aplicar a padronização no conjunto de teste
5.    Testar o modelo com o conjunto de teste





In [14]:
from sklearn.pipeline import Pipeline

scaler = StandardScaler()
model = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', model)])

n_splits = 10
cross_validator = StratifiedKFold(n_splits = n_splits, shuffle = True)
results = cross_validate(pipeline, features_sorted, target_sorted, cv = cross_validator, return_train_score=False)

print_results(results, n_splits)

Accuracy com cross validation de 10 splits: entre [74.71, 78.75]. 
        Média = 76.73 
        Desvio Padrão = 1.01


Apesar dessa mudança não se refletir significativamente nas métricas usadas, agora fizemos o certo e teremos mais garantia que o nosso modelo se comportará da mesma forma quando estiver em produção.